# **1-3. 스토킹 잠재/가해자 분류 모델 학습**

---
> 1-2에서 학습한 설문조사 임베딩 모델을 기반으로 스토킹 잠재/가해자 분류 모델을 학습합니다.

> 해석을 최우선 목표로 하고 있기 때문에, 스토킹 잠재/가해 여부 판별에 있어 어떤 문항이 어느 정도 기여했는지, 어떤 문항의 어떤 단어들이 얼마나 기여했는지 Attention을 통해 학습니다.
---

In [ ]:
# 드라이브 내 Custom Module 및 .ipynb 파일, 그리고 학습 데이터를 저장한 Directory를 입력하세요.
# 설문조사는 총 11개 문항으로 이루어져 있습니다. 학습을 원하는 문항의 번호를 입력하세요.

DIRECTORY = "AI경진대회" # 여기를 변경하세요.

## **(1) 라이브러리 준비**

In [ ]:
# Google Colab을 기반으로 학습을 진행할 경우, BERT 계열의 모델을 사용하기 위해 필요한 라이브러리를 설치합니다.

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install torchmetrics

In [ ]:
import torch
import time
import os
import pickle
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gluonnlp as nlp
import matplotlib.pyplot as plt
from tqdm import tqdm
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertTokenizer, BertModel, AutoModel, AutoTokenizer
from kobert import get_pytorch_kobert_model

## **(2) Drive Mount 및 Custom Module 불러오기**

In [ ]:
# Drive Mount
from google.colab import drive
drive.mount('/content/gdrive')

# Directory 변경
path = "/content/gdrive/My Drive/" + DIRECTORY
os.chdir(path)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from dataset import STALK
from model import NewSE, Stalking
from train import train_siamese
from utils import make_weights

## **(3) 데이터셋 불러오기**

In [ ]:
# 데이터셋 준비

with open('./DATA/3. 잠재 및 가해 분류/train_aug.pickle', 'rb') as f:
    train = pickle.load(f)

with open('./DATA/3. 잠재 및 가해 분류/valid_aug.pickle', 'rb') as f:
    valid = pickle.load(f)

In [ ]:
# Train/Valid 데이터의 warning에 대해 Label Encoding

temp = []

for i in range(train.shape[0]):
    if (train.loc[i, 'warning'] == 1) or (train.loc[i, 'warning'] == 2):
        temp.append(1)
    else:
        temp.append(0)

train['warning'] = temp

temp = []

for i in range(valid.shape[0]):
    if (valid.loc[i, 'warning'] == 1) | (valid.loc[i, 'warning'] == 2):
        temp.append(1)
    else:
        temp.append(0)

valid['warning'] = temp

In [ ]:
# Train/Valid 데이터의 status에 대해 Label Encoding

temp = []

for i in range(train.shape[0]):
    if train.loc[i, 'status'] == '상':
        temp.append(2)
    elif train.loc[i, 'status'] == '하':
        temp.append(1)
    else:
        temp.append(0)

train['status'] = temp

temp = []

for i in range(valid.shape[0]):
    if valid.loc[i, 'status'] == '상':
        temp.append(2)
    elif valid.loc[i, 'status'] == '하':
        temp.append(1)
    else:
        temp.append(0)

valid['status'] = temp

## **(4) KcELECTRA, Tokenizer 불러오기**

In [ ]:
# KcELECTRA, Tokenizer

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
kc_model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")

## **(5) DataLoader 준비**

In [ ]:
# 매우 불균형한 데이터의 형태를 고려하여 Train DataLoader에 Weighted Sampler 적용

weights = make_weights(train['warning'], 2)
weights = torch.DoubleTensor(weights).to(dtype=torch.float32)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

In [ ]:
# Dataset

train_set = STALK(tokenizer=tokenizer, embed_model=kc_model, data=train, shuffle=False, device=device)
valid_set = STALK(tokenizer=tokenizer, embed_model=kc_model, data=valid, shuffle=False, device=device)

In [ ]:
# DataLoader

BATCH_SIZE=64

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, sampler=sampler)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

## **(6) 모델 생성**

In [ ]:
# Model (Classifier)

model = Stalking(device=device, model_path='./TRAINED_MODEL/SE', batch_size=64) 

## **(7) 모델 학습**

In [ ]:
# Train Model

model_aft, train_loss, valid_loss = train_stalking(train_dataloader=train_loader,
                                                   valid_dataloader=valid_loader,
                                                   valid=True,
                                                   model=model,
                                                   epochs=20,
                                                   optimizer=optim.Adam,
                                                   criterion=nn.BCELoss,
                                                   scheduler=optim.lr_scheduler.StepLR,
                                                   gamma=0.8,
                                                   step_size=2,
                                                   lr=8e-03,
                                                   coef=0.1,
                                                   device=device)

## **(8) 학습 결과 시각화**

In [ ]:
# Accuracy 시각화

plt.figure(figsize=(12, 6))
plt.plot(train_loss)
plt.plot(valid_loss)
plt.title('Loss History')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(True)

plt.tight_layout()
plt.show()